# PYTHON_ADV - Part 2

In [ ]:
print("Welcome to PYTHON_ADV - Part 2")

### How to define class in Python
- *class* keyword
- as instance of *type*

In [ ]:
# Classic class definition

class Person:
    pass

p = Person()

# Dynamic class definition
# type(classname, parents(tuple), methods(dictionary))

Person = type('Person', (object,), {})

p = Person()

# Classic class definition

class Person:
    def __init__(self,jmeno):
        self.jmeno = jmeno
        
# Dynamic class definition

def myinit(self, name):
    self.name = name

Person = type('Person', (object,), {'__init__': myinit})

### Metaclasses
- objects are instances of classes
- classes are instances of metaclasses

<img src="https://th.bing.com/th/id/R.65d2a7c41f6962eed773c6ca9c45f2d5?rik=o1j86s%2flXiFHCg&riu=http%3a%2f%2fi.stack.imgur.com%2fQQ0OK.png&ehk=aJUnmCVdto%2fbaTZULT57HfbWgqR%2bWO1PVM3hphVmcls%3d&risl=&pid=ImgRaw&r=0"/>

In [26]:
class Controller(type):
    def __new__(cls, name, parents, content): # Contructor of classes
        print('Creating class',name)
        if len(name) <= 3:
            raise TypeError('Class name is too short')
        for name,value in content.items():
            if callable(value):
                print('Found method:',name)
        print('Number of parents is',len(parents))
        
        if len(parents) > 1:
            raise TypeError('Multiple inheritance is not supported')
        
        return super().__new__(cls, name, parents, content)
    
class User(metaclass=Controller):
    def __init__(self):
        pass
    def print(self):
        pass
    def __sub__(self,number):
        pass
    
class Employee(User):
    pass

class Next(str,User):
    pass

Creating class User
Found method: __init__
Found method: print
Found method: __sub__
Number of parents is 0
Creating class Employee
Number of parents is 1
Creating class Next
Number of parents is 2


TypeError: Multiple inheritance is not supported

In [ ]:
class MetaClass(type):
    def __new__(cls, *args, **kwargs):
        return type.__new__(cls, *args, **kwargs)
    
class Person(metaclass=MetaClass):
    def __init__(self, name):
        self.name = name
    def print(self):
        print(f'Name is {self.name}')

In [29]:
def fn(*args, **kwargs):
    print(args)
    print(kwargs)
    pass

fn()
fn(h='28')
fn(1,3,2,4)
fn(1,2,3,h=23,delta='d')

()
{}
()
{'h': '28'}
(1, 3, 2, 4)
{}
(1, 2, 3)
{'h': 23, 'delta': 'd'}


### Quick exercise
- Create metaclass 'ChildProtector'
- Metaclass requires that all classes begin with an uppercase character ( NameError )
- Metaclass requires that all classes implement 'ident' method

In [40]:
import string

class ChildProtector(type):
    def __new__(cls, name, parents, content):
        
        if name[0] not in string.ascii_uppercase:
            raise ValueError('Invalid class name')
            
        valid = False    
        
        for name,value in content.items():
            if name == 'ident' and callable(value):
                valid = True
                
        if not valid:
            raise ValueError('There is a need for ident method')
        
        return super().__new__(cls, name, parents, content)
    
class Help(metaclass=ChildProtector):
    def ident(self):
        pass
    
class Xelp(Help):
    pass

ValueError: There is a need for ident method

### Working with code
- Use code as data

In [47]:
# __code__ attribute

def add(a,b):
    'Functions sums two numbers'
    return a + b

def neg(a):
    return -a

def execute(code, a, b):
    if not callable(code):
        raise TypeError('not executable')
        
    if code.__code__.co_argcount != 2:
        raise TypeError('invalid number of arguments')
        
    print('calling function:',code.__name__)
        
    return code(a, b)

print(execute(add, 1,8))
print(execute(lambda x,y:x-y, 1, 8))
print(execute(neg, 1, 8))

calling function: add
9
calling function: <lambda>
-7


TypeError: invalid number of arguments

In [48]:
# Functions can be in collections

registry = dict()

def add(a,b):
    return a + b

class Multiply:
    def __call__(self, a, b):
        return a * b

registry['add'] = add                 # Function
registry['sub'] = lambda a,b: a - b   # Lambda
registry['mul'] = Multiply()          # Functor

print(registry)

for key in registry.keys():
    print(f'Calling function {key}: {registry[key](2,4)}')


{'add': <function add at 0x000002A49CF6F130>, 'sub': <function <lambda> at 0x000002A49D01F520>, 'mul': <__main__.Multiply object at 0x000002A49B4E2BC0>}
Calling function add: 6
Calling function sub: -2
Calling function mul: 8


### Quick exercise
- Create function register(registry, code)
- 'registry' is a dictionary
- 'code' must be a function with exactly two parameters !!! ( ValueError otherwise )
- 'register' append fuction 'code' to 'registry'. If 'code' is already there, then print warning
- list all fuctions from 'registry' in alphabet order

### Types in Python
- metadata about fuction parameter's types and return values
- *typing* module - type hints

In [49]:
def add(a: int, b: int) -> int:
    'Function sums two numbers'
    return a + b

print(add.__annotations__)

print(add(1,10))         # types are OK
print(add('abc',True))   # types are not OK, but still executable

import typing

Vector = typing.List[float]    # List of floating point numbers

def scale(scalar: float, vstup: Vector) -> Vector:
    return vstup

scale(2.0, [1.0, 3.4, 5.6])

UserId = typing.NewType('UserId', int) # We can define new types
some_id = UserId(12342)

{'a': <class 'int'>, 'b': <class 'int'>, 'return': <class 'int'>}
11


TypeError: can only concatenate str (not "bool") to str

In [54]:
def add(a,b):
    return a + b

def neg(a):
    return -a

def call(code, *numbers):
    return code(*numbers)

print(call(add,1,2))
print(call(neg,5))

3
-5


### Nested functions and closures
- nested function - function defined inside another function
- closure - function that remembers variables

In [55]:
def primary(somedata):                             # generic function
    """primary function"""
    print("primary: called")
    def nested():                                  # nested function
        """nested function"""
        print("nested: called, data =", somedata)
    nested()

primary("Hello")

primary: called
nested: called, data = Hello


In [56]:
def primary(somedata):                             
    """primary function"""
    def nested():
        """nested function"""
        print("nested data =", somedata)
        def supernested():
            """supernested function"""
            print("supernested: data =", somedata)
        supernested()
    nested()
    
primary("Hello")

nested data = Hello
supernested: data = Hello


In [18]:
def init(somedata):                          # generic function
    def nested(nextdata):                    # closure - somedata variable can be used here
        return somedata + ":" + nextdata
    return nested                            

enc = init("Hello")    # somedata = 'Hello'                      
print(enc("Python"))   # nextdata = 'Python'                       
print(enc("Perl"))     # nextdata = 'Perl'                     

Hello:Python
Hello:Perl


In [67]:
number = 8 # Heap - global

def fn():
    x = 28  # local
    global number
    number += 100
    print(number)
    
fn()

108


In [ ]:
x = 188 # Global

def init(start,step):                        # initialization function
    """Init function"""
    zz = 28 # nonlocal
    def get():  # closure
        yy = 28             # Local
        global start
        start += step                        # try to modify 'start': NOT ALLOWED !!!
        return start
    return get

d = init(10,2)
print(d())

In [71]:
def init(start,step):                        # initialization function
    """init function"""
    def better_get():                        # closure
        """closure"""
        nonlocal start                       # 'unlock' variable
        start += step                        # now we can modify it
        return start
    return better_get

enc = init(10,3)                             
print(enc())                                 # 13
print(enc())                                 # 16
print(enc())

13
16
19


In [61]:
def factory(operation):
    def plus(a,b):
        return a + b
    def minus(a,b):
        return a - b
    reg = {'+': plus, '-': minus}
    return reg[operation]

p = factory('-')
print(p(1,2))
print(p(10,8))

plus = factory('+')
print(plus(1,2))

-1
2
3


### Quick exercise
- Create closure that decorates strings

In [77]:
def decorator(prefix, suffix):
     def internal(string):
            return f'{prefix} {string}({len(string)}) {suffix}'
     return internal
    
dec = decorator('>>>','<<<')

print(type(dec))
print(dec.__name__)
print(dec.__code__.co_argcount)

print(dec('PYTHON')) # >>> PYTHON(6) <<<
print(dec('PERL')) # >>> PERL(4) <<<

<class 'function'>
internal
1
>>> PYTHON(6) <<<
>>> PERL(4) <<<


In [80]:
def fn(number):       # Ordinary function
    print(number)
    
def gn(code):         # high-order function
    return code()

def hn(code1, code2): # high-order function
    return code1() + code2()
    
fn(23)

gn(lambda:'hello')
hn(lambda:'abc', lambda:'xyz')

23


'abcxyz'

### Olegeech: higher order function example


In [17]:
def outer(fn):
  def inner(arg):
    print(f"Function '{fn.__name__}' was called with '{arg}'")
    fn(arg)
  return inner

def hello(user):
  print("Hello", user)

hello = outer(hello)
hello("Vasya!")


Function 'hello' was called with 'Vasya!'
Hello Vasya!


### Functors
- You can use functors instead of closures

In [ ]:
class TestFunctor:
    def __init__(self,start,step):
        self.start = start
        self.step  = step
        
    def __call__(self):
        tmpval = self.start
        self.start += self.step
        return tmpval
    
tf = TestFunctor(10,20)              # create functor

if callable(tf):                     # is callable ? YESSSS
    print("Ok,",tf," is callable")

print(tf())                          # call it - 10
print(tf())                          # call it - 30

for c in tf:
    print(c)


In [ ]:
def add(a,b):
    print('running add')
    return a + b

def sub(a,b):
    print('running sub')
    return a - b

### Decorators
- decorator - function that modified/impoves another function

#### Decorators without parameters

In [106]:
from functools import wraps

def debug(fn): # high-order function
#    @wraps(fn)
    def internal(*args, **kwargs):
        print('DEBUG: start')
        ret = fn(*args, **kwargs)
        print('DEBUG: stop')
        return ret
    internal.__name__ = fn.__name__
    return internal

@debug # greeting = debug(greeting)
def greeting():
    print('Hello from Python!')
    

def process(function):
    print('Running function',function.__name__)
    function()
    
process(greeting)

Running function greeting
DEBUG: start
Hello from Python!
DEBUG: stop


In [105]:
def runit(fn):
    fn()
    return fn

@runit # greeting = runit(greeting)
def greeting():
    print('Hello from Python!')
    
greeting()

Hello from Python!
Hello from Python!


#### Quick exercise
- Create decorator 'twice' that runs function two times
- In decorated fucntion there will be 'count' variable with value of 2

In [111]:
def twice(fn):
    fn()
    fn()
    
    def internal():
        print('from internal')
    
    fn.count = 2
    fn.code = internal
    return fn

@twice # fn = twice(fn)
def greeter():
    print('hello')
    
print(greeter.count) # 2
greeter.code()

greeter()

hello
hello
2
from internal
hello


#### Decorators with parameters

In [115]:
from functools import wraps
def fix(delta): # Ordinary function - 
    def internal(fn): # High order function
        @wraps(fn)
        def subinternal(a,b): # New version of fn
            return fn(a,b) + delta
        return subinternal
    return internal

def simplefix(fn): # High order function
    @wraps(fn)
    def internal(a,b): # New version of fn
        return fn(a,b) + 20
    return internal

#   f = fix(10)  f -> internal
#   sub = f(fn)  sub -> subiternal
#

@fix(10)           # sub = fix(10)(sub)
def sub(a,b):      # sub(a,b) + 10
    return a - b

#@simplefix
#def sub(a,b):
#    return a - b

print(sub.__name__)

print(sub(10,5)) # 10 - 5 + 10 = 15

sub
15


In [118]:
register = {}

def registerFunction(func):   
    print("Registering function:",func.__name__)
    register[func.__name__] = func
    return func

@registerFunction  # add = registerFunction(add)  
def add(a,b):
    return a + b

@registerFunction  # sub = registerFunction(sub)
def sub(a,b):
    return a - b

print(register)

result = sub(1,3)                                  
print("result =",result)


result = register['sub'](1,3)                        
print("result =",result)


{}
Registering function: sub
result = <function sub at 0x000002A49D0AF640>
result = -2


In [ ]:
import time

def timer(func):                           # useful decorator - stopwatch
    def internal(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        stop  = time.time()
        print("timer: function",func.__name__,"took", round(stop - start,4)," second(s)")
        return result
    return internal

@timer
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print(fib(6))


In [120]:
class Counter:                                 # Functor can be decorator, too :-)
    def __init__(self,func):
        self.func = func
        self.func.counter = 0
        
    def __call__(self,*args, **kwargs):
        self.func.counter += 1
        return self.func(*args, **kwargs)
    
    @property
    def counter(self):
        return self.func.counter
    
@Counter           # add = Counter(add)
def add(a,b):         
    return a + b

print(add(1,2))
print(add(3,4))

if hasattr(add, "counter"):
    print("yes, function is countable, #calls =",add.counter)

3
7
yes, function is countable, #calls = 2


#### Decorator on classes

In [135]:
def debug(fn):
    def internal(*args):
        print(f'DEBUG: calling {fn.__name__}')
        return fn(*args)
    return internal

class Math:
    @debug
    def add(self,a,b):
        return a + b
    
    @debug
    def sub(self,a,b):
        return a - b
    
    @debug
    def mul(self,a,b):
        return a * b
    
    @debug
    def div(self,a,b):
        return a / b
    
# Better solution - class decorator

def debugall(skipMagic=False):
    def internal(cls):
        for name,value in vars(cls).items():
            if callable(value):
                if skipMagic and name.startswith('__'):
                    print(f'Skipping magic method:',name)
                    continue
                print(f'Patching method: {name}')
                setattr(cls, name, debug(value))
        return cls
    return internal
    

@debugall()    # Math = debugall(Math)
class Math:
    def __init__(self):
        print('creating object')
        
    def __del__(self):
        print('destroying object')
        
    def add(self,a,b):
        return a + b
  
    def sub(self,a,b):
        return a - b
    
   
    def mul(self,a,b):
        return a * b
     
    def div(self,a,b):
        return a / b
    
    
    
m = Math()
print(type(Math))
print(type(m))
m.add(1,28)
del m

Patching method: __init__
Patching method: __del__
Patching method: add
Patching method: sub
Patching method: mul
Patching method: div
DEBUG: calling __init__
creating object
<class 'type'>
<class '__main__.Math'>
DEBUG: calling add
DEBUG: calling __del__
destroying object


In [126]:
def decor(cls):
    print('Decorating class',cls.__name__)
    for key,value in vars(cls).items():
        if callable(value):
            print('Found method',key)
    return cls

@decor # Math = decor(Math)
class Math:
    def __init__(self):
        print('object initialization ...')
    def add(self,a,b):
        return a + b
    def mul(self,a,b):
        return a * b
    
m = Math()
print(m.add(1,2))

Decorating class Math
Found method __init__
Found method add
Found method mul
object initialization ...
3


In [140]:
class Math:
    
    @staticmethod
    def add(a,b):
        return a + b
    
    #add = staticmethod(add)

    @classmethod
    def info(cls):
        pass
    
    #info = classmethod(info)
    
print(Math.add(1,2))
Math.info()

3


In [147]:
def timer(fn):
    print('timer:',fn.__name__)
    def internal_from_timer(a,b):
        return fn(a,b)
    return internal_from_timer

def debug(fn):
    print('debugging',fn.__name__)
    def internal_from_debug(a,b):
        return fn(a,b)
    return internal_from_debug

@timer           # add = timer(debug(add))
@debug
def add(a,b):
    return a + b

debugging add
timer: internal_from_debug


### Recursion
- function that calls itself
- it is necessary to define end condition

Example: factorial

*N! = N*(N-1)!*

*0! = 1*          End condition

Example: sum of numbers

*SUMA(N) = N + SUMA(N-1)*

*SUMA(0) = 0*    End condition

In [148]:
def fact(n):
    return n if n <= 1 else n*fact(n-1) # Ternary operator

print(fact(10))

3628800


### Quick exercise
- define recursive function length(string) - functions returns string's length
- define recursive function palindrome(string) - functions checks if string is a palindrome

In [156]:
def length(string):
    '''
    length('') = 0
    '''
    return 0 if len(string) == 0 else 1 + length(string[1:])

def palindrome(string):
    if len(string) <= 1:
        return True
    return False if string[0] != string[-1] else palindrome(string[1:-1])

False

### Functional programming
- process data with functions without modifications
- no side effects ( idempotency )
- _functools_, _itertools_, _operator_ modules

In [157]:
data = "This is the line with some short and vvvvverrrry loooooonnnnnnnng words".split(" ")
print("len(data) =",len(data))

len(data) = 11


In [158]:
shortwords = filter(lambda word:len(word) <= 3, data)  # filter() selects data based on code       
print(list(shortwords))

class WordFilter:                                             
    def __init__(self,maxlen):
        self.maxlen = maxlen
        
    def __call__(self,word):
        return True if len(word) <= self.maxlen else False


shortwords = filter(WordFilter(3), data)                     
print(list(shortwords))

['is', 'the', 'and']
['is', 'the', 'and']


In [159]:
print(data)
newdata = map(lambda word:word + "(" + str(len(word)) + ")", data) # map() applies code to each element
print(list(newdata))

['This', 'is', 'the', 'line', 'with', 'some', 'short', 'and', 'vvvvverrrry', 'loooooonnnnnnnng', 'words']
['This(4)', 'is(2)', 'the(3)', 'line(4)', 'with(4)', 'some(4)', 'short(5)', 'and(3)', 'vvvvverrrry(11)', 'loooooonnnnnnnng(16)', 'words(5)']


In [9]:
from functools import reduce                                    # reduce

data = "abcdefg"

singlestring = reduce(lambda a,b:a + "," + b, data)             
print(singlestring)                                             # principle  

def fn():
    pass                                                                #         a,b,c -> a op b, c -> (a op b) op c

print(type(fn))
print(fn.__class__)

a,b,c,d,e,f,g
<class 'function'>
<class 'function'>


In [ ]:
def fib(n):                              # not very good, too expensive
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

for i in range(0,40):
    print("fib(",i,")=",fib(i))

In [ ]:
from functools import lru_cache          # LRU cache

    
@lru_cache(maxsize=10)                # LRU cache with 10 slots
def fib(n):                             # better now
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

for i in range(0,100):
    print("fib(",i,")=",fib(i))
    
print("cache stat:", fib.cache_info())  # get cache stats


In [ ]:
from functools import reduce
import operator

def add(a,b):                                 # helping function
    return a + b

class Add():                                  # helping functor
    def __call__(self,a,b):
        return a + b

numbers = list(range(1,10))                   # list of numbers 1 .... 9
total   = reduce(lambda a,b:a+b, numbers)     # reduce using lambda
total   = reduce(add, numbers)                # reduce using function
total   = reduce(Add(), numbers)              # reduce using functor
total   = reduce(operator.add, numbers)       # reduce using add function from operator module

### Logical expressions

In [ ]:
# Legacy

size = 18

if size == 18:
    print('ok')
else:
    print('not ok')

# Logical expression

print('ok' if size == 18 else 'not ok')

# and even shorter version
# 1 -> True ( second element )
# 0 -> False ( first element )

('ok','not ok')[size != 18]

### Data persistence
- How to store objects/data to persistent place
- Where to store data
  1. databases ( SQL, NoSQL )
  2. files ( serialization/deserialization )
  3. send to a different location over TCP/IP ( RPC, Socket API, ZeroMQ, Nanomsg )

#### Storing/Loading objects to/from file
- *pickle* module
- files will be binary

In [ ]:
family = {
    'father': {
        'age': 38,
        'name': 'Jirka'
    },
    'mother': {
        'age': 36,
        'name': 'Vera'
    },
    'children': [
        {
            'age': 12,
            'name': 'Petr'
        },
        {
            'age': 14,
            'name': 'Petra'
        }
    ]
}

In [ ]:
import pickle

# Serialization
with open('family.db', 'wb') as storage:
    pickle.dump(family, storage)
    
del family

family = dict()

# Deserialiazation
with open('family.db', 'rb') as storage:
    family = pickle.load(storage)
    
print(family)   

In [ ]:
data = {'a':100, 'b':200}
transport = pickle.dumps(data)
data = pickle.loads(transport)
print(data)

#### Persistent dictionary
- *shelve* module
- key/value file-based database

In [ ]:
import shelve

data = shelve.open('data.db')
data['one'] = 'jedna'

print(data.keys())

data.close()

#### Working with JSON
- *json* module
- interface similar to *pickle*
- only built-in types are supported 

In [ ]:
class Test():
    def __init__(self):
        self.x = 100
        self.file = open('datafile.txt')

data = Test()

import json

# Serialization
with open('data.json', 'w') as storage:
    json.dump(data.__dict__, storage)

# Deserialization
with open('data.json', 'r') as storage:
    data = json.load(storage)
       

#### Working with CSV
- *csv* module

In [ ]:
import csv

data = [1,2,3,4]

# Serialization
with open('data.csv', 'w') as storage:
    writer = csv.writer(storage, delimiter=',')
    writer.writerow(data)

# Deserialization
with open('data.csv', 'r') as storage:
    reader = csv.reader(storage, delimiter=',')
    for radek in reader:
        print(radek)
    

### Scripts
- Files with Python code ( with *.py* extension )
- command line parameters - sys.argv )
- *#!* - specify Python interpreter
- use script as a module
- use different I/O channels in scripts
- script return value

In [ ]:
#!/usr/bin/python3

def main():
    print('hello')
    
if __name__ == '__main__':      # Run main(), but only when used as a script, not a module
    main()

#### Example: print content of a file

In [ ]:
#!/usr/bin/python3
import sys, os

def usage():                       # Helper method
    'Helper method'
    print('Usage: reader.py filename', file=sys.stderr)

if __name__ != '__main__':        # Protect this script from being used as module
    print('Sorry, this is not a module', file=sys.stderr)
    sys.exit(1)

if len(sys.argv) != 2:            # Check command line arguments
    usage()
    sys.exit(1)
    
filename = sys.argv[-1]

if not os.path.exists(filename): # Check file presence
    print(f'File {filename} does not exist', file=sys.stderr)
    sys.exit(1)

with open(filename) as source:  # Process file
    print(filename.readlines(), end='')
